In [158]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, log_loss, make_scorer
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date
import datetime
from utilities import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
# df = pd.read_csv('data/train.csv')
(X_train, X_test, y_train, y_test), column_names = load_and_split_data()

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 58 columns):
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
homepage                 946 non-null object
imdb_id                  3000 non-null object
original_title           3000 non-null object
overview                 2992 non-null object
popularity               3000 non-null float64
poster_path              2999 non-null object
production_companies     2844 non-null object
production_countries     2945 non-null object
release_date             3000 non-null int64
runtime                  3000 non-null float64
spoken_languages         2980 non-null object
status                   3000 non-null object
tagline                  2403 non-null object
title                    3000 non-null object
Keywords                 2724 non-null object
cast                  

In [168]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
id              3000 non-null int64
budget          3000 non-null int64
popularity      3000 non-null float64
runtime         3000 non-null float64
release_date    3000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 117.3 KB


In [169]:
rf = RandomForestRegressor(n_estimators=100,
                            n_jobs=-1,
                            random_state=1)

In [170]:
random_forest_grid = {'max_depth': [10],
                      'max_features': ['sqrt', 'log2', None],
                      'min_samples_split': [2,4],
                      'min_samples_leaf': [1, 2, 4],
                      'bootstrap': [True, False],
                      'n_estimators': [10,20,40,80,160],
                      'random_state': [1]}

rf_gridsearch = GridSearchCV(RandomForestRegressor(),
                             random_forest_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='neg_mean_squared_error')

rf_gridsearch.fit(X_train, y_train)

print( "best parameters:", rf_gridsearch.best_params_ )

best_rf_model = rf_gridsearch.best_estimator_

Fitting 3 folds for each of 180 candidates, totalling 540 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   17.1s finished


best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 160, 'random_state': 1}


In [189]:
print(rf_gridsearch.best_params_)

{'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 160, 'random_state': 1}


In [171]:
best_rf_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [172]:
best_rf_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [173]:
y_hat = best_rf_model.predict(X_test)

In [174]:
mean_squared_error(y_test,y_hat)

5442402264745912.0

In [175]:
mean_squared_log_error(y_test,y_hat)

6.0809696267783275

In [185]:
best_rf_model.fit(X,y)

df_test = pd.read_csv('test.csv')
df_test['runtime'].fillna((df['runtime'].mean()), inplace=True)
df_test['release_date'] = pd.to_datetime(df['release_date'])
df_test['release_date'] = datetime.date.today().year

X_test = df_test[['id','budget', 'popularity', 'runtime','release_date']]

y_test_hat = best_rf_model.predict(X_test)

In [187]:
#Generate CSV
df_test['revenue']= y_test_hat
dft = df_test[['id','revenue']]
dft.to_csv('lr_results.csv',index=False)